# Librerias 

In [ ]:
import numpy as np  
import pandas as pd  
import matplotlib.pyplot as plt 
%matplotlib inline  
import json 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
import folium 

# Datasets

In [ ]:
import wget

url="https://infra.datos.gob.ar/catalog/modernizacion/dataset/7/distribution/7.29/download/localidades-censales.csv"
path="C:/Users/u605929/Documents/CURSOS/PYTHON/Ciencia de Datos Aplicada Programa Especializado/4. Capstone/Batalla de los Vecindarios/"
wget.download(url,path)
url_internet="http://api.datosabiertos.enacom.gob.ar/api/v2/datastreams/CONEC-AL-SERVI-DE-INTER/data.csv?auth_key=44a38fbffd39c9f7d84e8e7dd2e1d02f0950e611&download=1"
wget.download(url_internet,path)
localidades=pd.read_csv("localidades-censales.csv")
internet=pd.read_csv("ConectividadalserviciodeInternet.csv")
provincias=pd.read_csv(path+"provincias.csv")

# Variables

In [ ]:
# CLIENT_ID = 'LM5HN3042JGPC50VLXUT2XG4L1Z4BN3DY1S5MBA5ANSVO54V' # su ID de Foursquare
CLIENT_SECRET = 'YCBLJNJX2WJMP0CKANFLTPXGSAQIARRGUKPB5BRROGAWPJO3' # Secreto de Foursquare
VERSION = '20180605' # versión de la API de Foursquare
LIMIT = 100 # Un valor límite para la API de Foursquare

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
cos_venues = getNearbyVenues(names=costa_data['nombre'],
                                   latitudes=costa_data['centroide_lat'],
                                   longitudes=costa_data['centroide_lon']
                                  )
costa_onehot = pd.get_dummies(cos_venues[['Venue Category']], prefix="", prefix_sep="")

costa_onehot['Neighbourhood'] = cos_venues['Neighbourhood'] 

fixed_columns = [costa_onehot.columns[-1]] + list(costa_onehot.columns[:-1])
costa_onehot = costa_onehot[fixed_columns]

costa_grouped = costa_onehot.groupby('Neighbourhood').mean().reset_index()
costa_grouped.head()
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
Se genera el nuevo dataframe y con los 3 sitios mas populares de cada localidad.

num_top_venues = 3

indicators = ['st', 'nd', 'rd']

columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = costa_grouped['Neighbourhood']

for ind in np.arange(costa_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(costa_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


#para la conexión 
acceso=internet.drop(["Partido","link","Latitud","Longitud"],axis=1)
int_var=acceso[(acceso["Provincia"]=="BUENOS AIRES")|
            (acceso["Provincia"]=="CHUBUT")|
            (acceso["Provincia"]=="SANTA CRUZ")|
            (acceso["Provincia"]=="RIO NEGRO")|
            (acceso["Provincia"]=="TIERRA DEL FUEGO")
            ]
variables_data=costa_grouped[["Neighbourhood"]].merge(int_var,how="left",left_on="Neighbourhood",right_on="Localidad")
variables_data.columns

from numpy import asarray
from sklearn.preprocessing import OrdinalEncoder

data = variables_data[['ADSL','CABLEMODEM','DIALUP','FIBRAOPTICA','SATELITAL','WIRELESS', 'TELEFONIAFIJA','3G','4G']]

encoder = OrdinalEncoder()

result = encoder.fit_transform(data)
Convertir de nuevo a un dataframe y unirlo a las variables geográficas ya normalizadas


internet_grouped=pd.DataFrame(data=result,columns=[data.columns]).reset_index(drop=True)
var_grouped=pd.concat([costa_grouped,internet_grouped], axis=1)
internet_grouped.describe()

internet_data=var_grouped.loc[:,var_grouped.columns[[0] +list(range(76, var_grouped.shape[1]))]]

internet_data["Red_Movil"]=np.where(internet_data.iloc[:,8]+internet_data.iloc[:,9]>0,"SI","NO")
internet_data["Red_BandaAncha"]=np.where(internet_data.iloc[:,1]+internet_data.iloc[:,2]+internet_data.iloc[:,4]>0,"SI","NO")
internet_data["Red_BandaAngosta"]=np.where(internet_data.iloc[:,3]+internet_data.iloc[:,5]+internet_data.iloc[:,6]>0,"SI","NO")
internet_data["Telefono_Fijo"]=np.where(internet_data.iloc[:,7]>0,"SI","NO")
acceso_data=internet_data.loc[:,internet_data.columns[[0] + list(range(10,internet_data.shape[1]))]]
acceso_data.insert(1,'Poblacion', variables_data["Poblacion"])
acceso_data.head()




# Clustering 

In [ ]:

costa_grouped_clustering = var_grouped.drop('Neighbourhood', 1)

X=costa_grouped_clustering
Nc = range(1, 20)
kmeans = [KMeans(n_clusters=i) for i in Nc]
kmeans
score = [kmeans[i].fit(X).score(X) for i in range(len(kmeans))]
score
plt.plot(Nc,score)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Curve')
plt.show()
kclusters = 3

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(costa_grouped_clustering)
kmeans.labels_[0:10] 
costa_localidades=costa_data[["Localidades_Costa","provincia_nombre","centroide_lat","centroide_lon"]].merge(acceso_data,left_on='Localidades_Costa', right_on='Neighbourhood')
costa_localidades=costa_localidades.drop("Neighbourhood",axis=1)
costa_localidades.head()
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=4)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(costa_merged['centroide_lat'], costa_merged['centroide_lon'], costa_merged['Localidades_Costa'], costa_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters
